In [10]:
import webvtt
from os import listdir
from os.path import isfile, join
import pandas as pd
from pydub import AudioSegment
import os
import shutil
import natsort 
import wave
import xml.etree.ElementTree as ET

In [12]:
rootdir= "/home/megha/Desktop/b"

# 1) # make new directory and convert the wav file
#---------------------------------------------------------------------------------------------------------------
if False: #False  True
    i= 0
    for Dir in natsort.natsorted(os.listdir(rootdir)):
        if not os.path.exists(rootdir+"/"+Dir+"/temp"): 
            os.makedirs(rootdir+"/"+Dir+"/temp")
        i = i+1
        for file in natsort.natsorted(os.listdir(rootdir+"/"+Dir)):
            if(file.endswith(".WAV")):
                actual_filename = "folk_"+str(i)
                a = os.system('ffmpeg -i "{}" -acodec pcm_s16le -ac 1 -ar 16000 "{}/{}".wav'.format(rootdir+"/"+Dir+"/"+file, rootdir+"/"+Dir+"/temp", actual_filename))# (source,target(half path),name_of_file)
                print(a)
                if(a==256):
                        print("Did not get converted")
            if(file.endswith(".fln")):
                shutil.copyfile(rootdir+"/"+Dir+"/"+file, rootdir+"/"+Dir+"/temp/"+file)
            
#2) extract information from .fln file
#---------------------------------------------------------------------------------------------------------------
if False: #False  True
    for Dir in natsort.natsorted(os.listdir(rootdir)):        
        fileName=""
        #df  = pd.DataFrame(columns = ['A', 'B', 'C'])
        df1  = pd.DataFrame(columns = ['A', 'B', 'C'])
        df2  = pd.DataFrame(columns = ['A', 'B', 'C'])
        df3  = pd.DataFrame(columns = ['A', 'B', 'C'])
        for file in os.listdir(rootdir+"/"+Dir+"/temp"):            
            if(file.endswith(".wav")):
                fileName = file.split('.')[-2]+"_"
        for file in natsort.natsorted(os.listdir(rootdir+"/"+Dir+"/temp")):
            #print(file)
            if(file.endswith(".fln")):                
                if(file.endswith(".fln") and file.split('_')[-1]=="01.fln" ):
                    tree = ET.parse(rootdir+"/"+Dir+"/temp/"+file)
                    root = tree.getroot()
                    for contribution in root.findall("./contribution"):
                        children = contribution.getchildren()
                        time_array = []  
                        word = ""
                        point = 0
                        for child in children:   
                            if(child.tag == "time"):
                                if(point == 0):
                                    time_array.append(child.attrib['time'])
                                if(point == 1):
                                    time_array.append(child.attrib['time'])
                                    df_a = pd.DataFrame([[time_array[0],time_array[-1],word]], columns=list('ABC'))                            
                                    df1 = df1.append(df_a,ignore_index = True)
                                    word = ""
                                    time_array[0] = time_array[-1]  
                                    point = 0
                            if(child.tag == "uncertain"):
                                for c in child:
                                    word = word+" "+c.text
                            if(child.tag == "w"):
                                word = word+" "+child.text
                            if(child.tag == "p"):
                                point = 1
                                continue                                             
                #print(df1.size)
                if(file.endswith(".fln") and file.split('_')[-1]=="02.fln" ): 
                    tree = ET.parse(rootdir+"/"+Dir+"/temp/"+file)
                    root = tree.getroot()
                    for contribution in root.findall("./contribution"):
                        children = contribution.getchildren()
                        time_array = []  
                        word = ""
                        for child in children:   
                            if(child.tag == "time"):
                                if(point == 0):
                                    time_array.append(child.attrib['time'])
                                if(point == 1):
                                    time_array.append(child.attrib['time'])
                                    df_b = pd.DataFrame([[time_array[0],time_array[-1],word]], columns=list('ABC'))                            
                                    df2 = df2.append(df_b,ignore_index = True)
                                    word = ""
                                    time_array[0] = time_array[-1]  
                                    point = 0
                            if(child.tag == "uncertain"):
                                for c in child:
                                    word = word+" "+c.text
                            if(child.tag == "w"):
                                word = word+" "+child.text
                            if(child.tag == "p"):
                                point = 1
                                continue  
                if(file.endswith(".fln") and file.split('_')[-1]=="03.fln" ): 
                    tree = ET.parse(rootdir+"/"+Dir+"/temp/"+file)
                    root = tree.getroot()
                    for contribution in root.findall("./contribution"):
                        children = contribution.getchildren()
                        time_array = []  
                        word = ""
                        for child in children:   
                            if(child.tag == "time"):
                                if(point == 0):
                                    time_array.append(child.attrib['time'])
                                if(point == 1):
                                    time_array.append(child.attrib['time'])
                                    df_c = pd.DataFrame([[time_array[0],time_array[-1],word]], columns=list('ABC'))                            
                                    df3 = df3.append(df_c,ignore_index = True)
                                    word = ""
                                    time_array[0] = time_array[-1]  
                                    point = 0
                            if(child.tag == "uncertain"):
                                for c in child:
                                    word = word+" "+c.text
                            if(child.tag == "w"):
                                word = word+" "+child.text
                            if(child.tag == "p"):
                                point = 1
                                continue  
                #print(df3.size) 
        frames = []
        frames = [df1, df2, df3]
        df = pd.concat(frames)
        df.reset_index(drop=True, inplace=True)
        #check = df.iloc[::1, :].index
        time1 = []
        time2=  []
        starttime = []
        endtime = []
        string = []
        string_final = []
        for i in range(len(df)):
            x = df.get_values()[i]
            time1.append(float(x[0]))
            time2.append(float(x[1]))
            string.append(x[2])
            starttime.append(time1[-1])
            endtime.append(time2[-1])
            string_final.append(string[-1]) 
        string_final =  [x.upper() for x in string_final] 
        df_final = pd.DataFrame({'col1':starttime, 'col2':endtime, 'col3':string_final})
        df_final = df_final.drop(df_final[df_final.col3 ==""].index)         
        filename_list = []
        for i in range(0, len(df_final)):                    
            filename_list.extend([fileName+str(i)])    
        df_final['col4'] = filename_list
        print(df_final.shape)
        #----------------------------txt file creation-----------------------------------------------------------------   
        with open(rootdir+"/"+Dir+"/temp/"+"folk.trans.txt", "a") as file:
            for i in range(0, len(df_final)):
                file.write(df_final['col4'].iloc[i]+" "+df_final['col3'].iloc[i]+"\n")
        #----------------------------chuncking-----------------------------------------------------------------   
        for file1 in natsort.natsorted(os.listdir(rootdir+"/"+Dir+"/temp/")):
            if(file1.endswith(".wav")):
                myaudio = AudioSegment.from_file(rootdir+"/"+Dir+"/temp/"+file1, "wav")
                for i in range(0, len(df_final)):
                    START = (df_final['col1'].iloc[i] * 1000)
                    END = (df_final['col2'].iloc[i] * 1000)
                    chunk = myaudio[START:END]     
                    chunk_name = df_final['col4'].iloc[i]+".flac" 
                    if not os.path.exists(rootdir+"/"+Dir+"/temp/"+chunk_name):
                        print("chunking the audio", chunk_name)
                        chunk.export(rootdir+"/"+Dir+"/temp/"+chunk_name, format="wav")
            
#3) --------------------------------------cleaning folder---------------------------------------------
if False: # False True
    for Dir in natsort.natsorted(os.listdir(rootdir)):
        #print(Dir)
        for file in natsort.natsorted(os.listdir(rootdir+"/"+Dir)):
            print(file)
            if(file.endswith(".xml") or file.endswith(".WAV") or file.endswith(".fln") or file.endswith(".html")):
                os.remove(rootdir+"/"+Dir+"/"+file)
        for file in natsort.natsorted(os.listdir(rootdir+"/"+Dir+"/temp")):
            print(file)
            if(file.endswith(".fln") or file.endswith(".wav")):
                os.remove(rootdir+"/"+Dir+"/temp/"+file)
            else:
                shutil.move(rootdir+"/"+Dir+"/temp/"+file, rootdir+"/"+Dir+"/"+file)
        os.rmdir(rootdir+"/"+Dir+"/temp/")


ZW--_E_00601.xml
ZW--_E_00601_SE_01_A_02_DF_01.WAV
ZW--_E_00601_SE_01_T_01.fln
ZW--_E_00601_SE_01_T_01.html
ZW--_E_00601_e_doc.html
ZW--_S_00702.xml
temp
ZW--_E_00601_SE_01_T_01.fln
folk.trans.txt
folk_1.wav
folk_1_0.flac
folk_1_1.flac
folk_1_2.flac
folk_1_3.flac
folk_1_4.flac
folk_1_5.flac
folk_1_6.flac
folk_1_7.flac
folk_1_8.flac
folk_1_9.flac
folk_1_10.flac
folk_1_11.flac
folk_1_12.flac
folk_1_13.flac
folk_1_14.flac
folk_1_15.flac
folk_1_16.flac
folk_1_17.flac
folk_1_18.flac
folk_1_19.flac
folk_1_20.flac
folk_1_21.flac
folk_1_22.flac
folk_1_23.flac
folk_1_24.flac
folk_1_25.flac
folk_1_26.flac
folk_1_27.flac
folk_1_28.flac
folk_1_29.flac
folk_1_30.flac
folk_1_31.flac
folk_1_32.flac
folk_1_33.flac
folk_1_34.flac
folk_1_35.flac
folk_1_36.flac
folk_1_37.flac
folk_1_38.flac
folk_1_39.flac
folk_1_40.flac
folk_1_41.flac
folk_1_42.flac
folk_1_43.flac
folk_1_44.flac
folk_1_45.flac
folk_1_46.flac
folk_1_47.flac
folk_1_48.flac
folk_1_49.flac
folk_1_50.flac
folk_1_51.flac
folk_1_52.flac
folk_1_

ZW--_E_02515_SE_01_T_01.fln
ZW--_E_02515_SE_01_T_01.html
ZW--_E_02515_e_doc.html
ZW--_S_02652.xml
temp
ZW--_E_02515_SE_01_T_01.fln
folk.trans.txt
folk_6.wav
folk_6_0.flac
folk_6_1.flac
folk_6_2.flac
folk_6_3.flac
folk_6_4.flac
folk_6_5.flac
folk_6_6.flac
folk_6_7.flac
folk_6_8.flac
folk_6_9.flac
folk_6_10.flac
folk_6_11.flac
folk_6_12.flac
folk_6_13.flac
folk_6_14.flac
folk_6_15.flac
folk_6_16.flac
folk_6_17.flac
folk_6_18.flac
folk_6_19.flac
folk_6_20.flac
folk_6_21.flac
folk_6_22.flac
folk_6_23.flac
folk_6_24.flac
folk_6_25.flac
folk_6_26.flac
folk_6_27.flac
folk_6_28.flac
folk_6_29.flac
folk_6_30.flac
folk_6_31.flac
folk_6_32.flac
folk_6_33.flac
folk_6_34.flac
folk_6_35.flac
folk_6_36.flac
folk_6_37.flac
folk_6_38.flac
folk_6_39.flac
folk_6_40.flac
folk_6_41.flac
folk_6_42.flac
folk_6_43.flac
folk_6_44.flac
folk_6_45.flac
folk_6_46.flac
folk_6_47.flac
folk_6_48.flac
folk_6_49.flac
folk_6_50.flac
folk_6_51.flac
folk_6_52.flac
folk_6_53.flac
folk_6_54.flac
folk_6_55.flac
folk_6_56.fla

folk_13_27.flac
folk_13_28.flac
folk_13_29.flac
folk_13_30.flac
folk_13_31.flac
folk_13_32.flac
folk_13_33.flac
folk_13_34.flac
folk_13_35.flac
folk_13_36.flac
folk_13_37.flac
folk_13_38.flac
folk_13_39.flac
folk_13_40.flac
folk_13_41.flac
folk_13_42.flac
folk_13_43.flac
folk_13_44.flac
folk_13_45.flac
folk_13_46.flac
folk_13_47.flac
folk_13_48.flac
folk_13_49.flac
folk_13_50.flac
folk_13_51.flac
folk_13_52.flac
folk_13_53.flac
folk_13_54.flac
folk_13_55.flac
folk_13_56.flac
folk_13_57.flac
folk_13_58.flac
folk_13_59.flac
folk_13_60.flac
folk_13_61.flac
folk_13_62.flac
folk_13_63.flac
folk_13_64.flac
folk_13_65.flac
folk_13_66.flac
folk_13_67.flac
folk_13_68.flac
folk_13_69.flac
folk_13_70.flac
folk_13_71.flac
folk_13_72.flac
folk_13_73.flac
folk_13_74.flac
folk_13_75.flac
folk_13_76.flac
folk_13_77.flac
folk_13_78.flac
folk_13_79.flac
folk_13_80.flac
folk_13_81.flac
folk_13_82.flac
folk_13_83.flac
folk_13_84.flac
folk_13_85.flac
folk_13_86.flac
folk_13_87.flac
folk_13_88.flac
folk_13_

In [34]:
#df = df[['A','B', 'C']]
df.reset_index(drop=True, inplace=True)



In [8]:
pd.set_option('display.max_colwidth', -1)
df_final

,col1,col2,col3,col4
0,0.51,0.720,JA,folk_1_0
1,0.72,3.540,ICH MÖCHTE JA NUN MAL GERNE VON MEINER HEIMAT ERZÄHLEN,folk_1_1
2,3.54,8.960,ICH BIN ZWAR SEIT NEUNZEHNHUNDERTNEUNUNDVIERZIG SCHON HIER IM RHEINLAND,folk_1_2
3,8.96,12.700,UND ES GEFÄLLT MIR JA AUCH SEHR SCHÖN UND SEHR GUT,folk_1_3
4,12.70,14.510,ABER DAHEIM IST EBEN DAHEIM,folk_1_4
5,14.51,18.800,GEBOREN BIN ICH IN OSCHATZ,folk_1_5
6,18.80,19.350,OSCHATZ,folk_1_6
7,19.35,20.930,DA MÜSSEN DIE MEISTEN LEUTE LACHEN,folk_1_7
8,20.93,21.600,WENN SIE DAS HÖREN,folk_1_8
9,21.60,25.620,DAS SOLL ANGEBLICH DAVON KOMMEN,folk_1_9


In [11]:
df.to_csv("/home/megha/Desktop/c/01.csv", sep='\t', encoding='utf-8')